In [ ]:
import torch

data = torch.load('../torchfem_dataset/panel_plasticity_2/simulation_dump_151.pt', weights_only=False)

In [1]:
import json
import torch
import numpy as np

def to_pure(obj):
    # Tensors (handles sparse/non-strided and meta)
    if isinstance(obj, torch.Tensor):
        t = obj.detach()
        if t.device.type == "meta":
            return {"_type": "tensor", "device": "meta", "dtype": str(t.dtype), "shape": list(t.shape)}

        # Non-strided layouts (sparse/nested/mkldnn)
        if getattr(t, "layout", torch.strided) != torch.strided:
            # Handle common sparse layouts explicitly
            if getattr(t, "is_sparse", False):
                layout = t.layout
                try:
                    if layout == torch.sparse_coo:
                        return {
                            "_type": "torch.sparse_coo_tensor",
                            "size": list(t.size()),
                            "indices": t._indices().detach().cpu().tolist(),
                            "values": t._values().detach().cpu().tolist(),
                        }
                    if hasattr(torch, "sparse_csr") and layout == torch.sparse_csr:
                        return {
                            "_type": "torch.sparse_csr_tensor",
                            "size": list(t.size()),
                            "crow_indices": t.crow_indices().detach().cpu().tolist(),
                            "col_indices": t.col_indices().detach().cpu().tolist(),
                            "values": t.values().detach().cpu().tolist(),
                        }
                    if hasattr(torch, "sparse_csc") and layout == torch.sparse_csc:
                        return {
                            "_type": "torch.sparse_csc_tensor",
                            "size": list(t.size()),
                            "ccol_indices": t.ccol_indices().detach().cpu().tolist(),
                            "row_indices": t.row_indices().detach().cpu().tolist(),
                            "values": t.values().detach().cpu().tolist(),
                        }
                    if hasattr(torch, "sparse_bsr") and layout == torch.sparse_bsr:
                        return {
                            "_type": "torch.sparse_bsr_tensor",
                            "size": list(t.size()),
                            "crow_indices": t.crow_indices().detach().cpu().tolist(),
                            "col_indices": t.col_indices().detach().cpu().tolist(),
                            "values": t.values().detach().cpu().tolist(),
                        }
                except Exception:
                    pass  # fall through to dense attempt

            # Fallback: try densifying
            try:
                return t.to_dense().cpu().tolist()
            except Exception:
                return {"_type": "tensor", "layout": str(t.layout), "size": list(t.size())}

        # Regular strided tensor
        t = t.cpu()
        try:
            return t.tolist()
        except RuntimeError as e:
            if "doesn't have storage" in str(e):
                try:
                    return t.to_dense().cpu().tolist()
                except Exception:
                    return {"_type": "tensor", "layout": str(t.layout), "size": list(t.size())}
            raise

    # Numpy arrays and scalars
    if np is not None and isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.generic,)):
        return obj.item()

    # Containers
    if isinstance(obj, dict):
        # JSON needs string keys; stringify to be safe
        return {str(k): to_pure(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [to_pure(v) for v in obj]

    # Torch meta-types
    if isinstance(obj, torch.Size):
        return list(obj)
    if isinstance(obj, (torch.dtype, torch.device)):
        return str(obj)

    # Primitives or fallback to string
    try:
        json.dumps(obj)
        return obj
    except Exception:
        return str(obj)


def convert_pt_to_json(pt_path, json_path):
    # Load the .pt file on CPU to avoid CUDA dependency
    data = torch.load(pt_path, map_location="cpu",weights_only=False)

    # Recursively purify it
    pure_data = to_pure(data)

    # Write to JSON file
    with open(json_path, "w") as f:
        json.dump(pure_data, f, indent=2)

    print(f"✅ Converted {pt_path} → {json_path}")

convert_pt_to_json("../torchfem_dataset/panel_plasticity_2/simulation_dump_155.pt", "../base/test_dataset/val5.json")

✅ Converted ../torchfem_dataset/panel_plasticity_2/simulation_dump_155.pt → ../base/test_dataset/val5.json
